In [1]:
import scipy
from scipy.io import wavfile
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal
import osascript
from gtts import gTTS 
import os 
import pyaudio
import wave
import keyboard  # using module keyboard
import soundfile as sf
import math
import pyloudnorm as pyln
from sys import byteorder
from array import array
from struct import pack
import librosa
from scipy.signal import butter, sosfiltfilt
import python_speech_features
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from ipynb.fs.full.Pitch_vector import get_pitch_stats
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 
import time
from ipynb.fs.full.Pitch_vector import get_lpc_stats
from ipynb.fs.full.Pitch_vector import get_spectral_stats
import ipynb.fs.full.concat_project2 as features_aron


Using TensorFlow backend.


In [2]:
BANDPASS_FREQ = [300, 3400]

In [3]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], btype='band', analog=False, output='sos')
    return sos

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfiltfilt(sos, data)
    return y

In [4]:
def normalize(snd_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM)/max(abs(i) for i in snd_data)

    r = array('h')
    for i in snd_data:
        r.append(int(i*times))
    return r

In [5]:
def extract_data_Ravdess(file_location):
    fs, data = wavfile.read(file_location)
    number_of_samples = data.shape[0]
    meta_data = open(r"LDC2002S28-txt.txt")
    meta_data = pd.read_csv("LDC2002S28-txt.txt", sep="A:", header=None, engine='python')
    meta_data.columns = ["sound limits","description"]
    
    #dual channel to one channel
    data = np.average(data, axis = 1)
    #remove noise
    data = butter_bandpass_filter(data, BANDPASS_FREQ[0], BANDPASS_FREQ[1], fs)
    
    # removee extra data pointts
    meta_data = meta_data[meta_data.description != ' [MISC]']
    meta_data = meta_data[~meta_data['description'].astype(str).str.startswith(' (')]
    meta_data = meta_data[~meta_data['description'].astype(str).str.startswith(' Emotion category elation')]
    meta_data = meta_data[~meta_data['description'].astype(str).str.startswith('  [MISC]')]

    # description and time limits 
    voice_time_limits = meta_data["sound limits"]
    voice_time_limits = [i.split(" ")[0:2] for i in voice_time_limits]
    voice_time_limits = np.array(voice_time_limits)
    voice_time_limits = voice_time_limits.astype(np.float)
    description = meta_data["description"]
    description = [i.split(",")[0].strip() for i in description]

    #divide the dataa set
    divided_data = []
    for i in voice_time_limits:
        startingpoint = int(i[0]*fs)
        endingpoint = int(i[1]*fs)
        divided_data.append(data[startingpoint:endingpoint])
    np_data = np.asarray(divided_data)
    return np_data, description, len(divided_data), fs

In [6]:
folder_location = '../../RAVDESS'
def data_extraction_RAVDESS(file_location):
    divided_data = []
    directory = os.fsencode(folder_location)
    for file_name in os.listdir(directory):
        if  str(file_name) != "b'.DS_Store'":
            (sig, rate) = librosa.load(folder_location+'/'+str(file_name)[2:-1], sr=None)
            data = butter_bandpass_filter(sig, BANDPASS_FREQ[0], BANDPASS_FREQ[1], rate)
            data = normalize(data)
            divided_data.append(data)
    np_data = np.asarray(divided_data)
    number_examples = np_data.shape[0]
    return divided_data, number_examples

In [43]:
def emotion_extraction_RAVDESS(folder_location, number_examples):
    nu_emotion = 8
    y = np.zeros(shape=(nu_emotion*2, number_examples))
    counter = 0
    directory = os.fsencode(folder_location)
    for file_name in os.listdir(directory):
        if  str(file_name) != "b'.DS_Store'":
            emotion = str(file_name)[2:-1]
            i = (emotion.split('-')[2])
            male = int(emotion.split('-')[-1].split('.')[0])%2
            if i == '01': # neutral 
                if male:
                    y[0][counter] = 1 # neutral male
                else:
                    y[1][counter] = 1
            elif i == '02': #calm
                if male:
                    y[2][counter] = 1 # neutral male
                else:
                    y[3][counter] = 1
            elif i == '03': #hahppy 
                if male:
                    y[4][counter] = 1 # neutral male
                else:
                    y[5][counter] = 1
            elif i == '04': #sad
                if male:
                    y[6][counter] = 1 # neutral male
                else:
                    y[7][counter] = 1
            elif i == '05': #angry
                if male:
                    y[8][counter] = 1 # neutral male
                else:
                    y[9][counter] = 1
            elif i == '06': #fearful
                if male:
                    y[10][counter] = 1 # neutral male
                else:
                    y[11][counter] = 1
            elif i == '07': #disgust
                if male:
                    y[12][counter] = 1 # neutral male
                else:
                    y[13][counter] = 1
            elif i == '08': #surprised
                if male:
                    y[14][counter] = 1 # neutral male
                else:
                    y[15][counter] = 1
            else:
                print(i)
                break
            counter +=1
    y = np.transpose(y)
    print(counter)
    return y

In [44]:
y = emotion_extraction_RAVDESS('../../RAVDESS', 1380)

1380


In [45]:
np.sum(y[:,5])

88.0